In [3]:
!pip install gym -U

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 36.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827627 sha256=12dada5d149d31699b64400a10ebc3bb56d1105a64bc41a0135eae82f745243a
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.6 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [2]:
import gym
import tensorflow as tf
import numpy as np
import random
from collections import deque
from skimage.color import rgb2gray
from skimage.transform import resize
import os
import matplotlib.pyplot as plt
import gc

def QNeuralNetwork(InputShape, OutputShape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=InputShape),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(OutputShape, activation='linear')
    ])
    return model

def PreProcess(frame):
    im = rgb2gray(frame)
    normalized_frame = im/255.0
    resized_frame = resize(normalized_frame, [110, 84])
    return np.float32(resized_frame)

EPSILON = 1.000
EPSILON_DECAY = 0.999
EPSILON_MIN = 0.001
GAMMA = 0.950
BATCH_SIZE = 64
LEARNING_RATE = 0.001
MEMORY_SIZE = 5000

states, actions, rewards, next_states, terminateds = deque(maxlen=MEMORY_SIZE), deque(maxlen=MEMORY_SIZE), deque(maxlen=MEMORY_SIZE), deque(maxlen=MEMORY_SIZE), deque(maxlen=MEMORY_SIZE)

env = gym.make("CartPole-v1")

state, info = env.reset()

model = QNeuralNetwork(env.observation_space.shape, env.action_space.n)
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='mse')

checkpoint_path = "training_2/my_model"
#model = tf.keras.models.load_model(checkpoint_path)

frame_count = 0
total_reward = 0
Total_episodes = 0
TimesObjectiveReached = 0

IsTesting = False

while True:
    frame_count += 1
    randNum = np.random.rand()
    action = 0
    if randNum > EPSILON or IsTesting:
        q_values = model(tf.convert_to_tensor(np.array([state]))).numpy()
        action = np.argmax(q_values[0])
    else:
        action = env.action_space.sample()

    next_state, reward, terminated, truncated, info = env.step(action)

    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    terminateds.append(terminated)
    state = np.copy(next_state)
    total_reward += reward

    if total_reward >= 500 and not IsTesting:
        print("Modelo chegou ao objetivo!")
        TimesObjectiveReached += 1
        if TimesObjectiveReached >= 3:
            model.save(checkpoint_path)
            break

    if len(states) > BATCH_SIZE and not IsTesting:
        batch_index = np.random.choice(len(states), BATCH_SIZE)
        batch_states = np.array([states[i] for i in batch_index])
        batch_actions = np.array([actions[i] for i in batch_index])
        batch_rewards = np.array([rewards[i] for i in batch_index])
        batch_next_states = np.array([next_states[i] for i in batch_index])
        batch_terminateds = np.array([terminateds[i] for i in batch_index])
        q_values = model(tf.convert_to_tensor(batch_states)).numpy()
        q_values_next = model(tf.convert_to_tensor(batch_next_states)).numpy()
        batch_index = np.arange(BATCH_SIZE, dtype=np.int32)
        q_target = np.copy(q_values)
        q_target[batch_index, batch_actions] = batch_rewards + GAMMA * np.max(q_values_next, axis=1) * (1 - batch_terminateds)
        model.fit(tf.convert_to_tensor(batch_states), tf.convert_to_tensor(q_target), epochs=1, verbose=1)
        # "Solve" memory leak
        #tf.keras.backend.clear_session()
        #gc.collect()

    if EPSILON > EPSILON_MIN:
        EPSILON *= EPSILON_DECAY
    if terminated or truncated:
        Total_episodes += 1
        print(f"Reward: {total_reward}, Total episodes: {Total_episodes}")
        total_reward = 0
        observation, info = env.reset()

env.close()

A saída de streaming foi truncada nas últimas 5000 linhas.
2/2 [==============================] - 0s 5ms/step - loss: 0.6069
Reward: 162.0, Total episodes: 167
2/2 [==============================] - 0s 8ms/step - loss: 0.0557
Reward: 169.0, Total episodes: 168
2/2 [==============================] - 0s 7ms/step - loss: 0.0521
Reward: 209.0, Total episodes: 169
2/2 [==============================] - 0s 8ms/step - loss: 0.0466
Reward: 322.0, Total episodes: 170
2/2 [==============================] - 0s 7ms/step - loss: 0.4092
Reward: 114.0, Total episodes: 171
2/2 [==============================] - 0s 4ms/step - loss: 0.0335
Reward: 304.0, Total episodes: 172
2/2 [==============================] - 0s 9ms/step - loss: 1.2726
Reward: 213.0, Total episodes: 173
2/2 [==============================] - 0s 6ms/step - loss: 0.0979
Reward: 241.0, Total episodes: 174
2/2 [==============================] - 0s 8ms/step - loss: 0.0802
Reward: 128.0, Total episodes: 175
2/2 [===========================

In [10]:
!zip -r file.zip ./training_2

  adding: training_2/ (stored 0%)
  adding: training_2/my_model/ (stored 0%)
  adding: training_2/my_model/fingerprint.pb (stored 0%)
  adding: training_2/my_model/saved_model.pb (deflated 88%)
  adding: training_2/my_model/variables/ (stored 0%)
  adding: training_2/my_model/variables/variables.index (deflated 59%)
  adding: training_2/my_model/variables/variables.data-00000-of-00001 (deflated 23%)
  adding: training_2/my_model/assets/ (stored 0%)
  adding: training_2/my_model/keras_metadata.pb (deflated 87%)
